In [37]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You have been asked to find a good restaurant in a city you are visiting."),
    ("human", "I want to cook {cuisine}."),
])

chef_chain = chef_prompt | chat

In [38]:
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it.",
        ),
        ("human", "{recipe}"),
    ]
)


veg_chain = veg_chefs_prompt | chat


final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({"cuisine": "indian"})

That's great! If you're looking for a good Indian restaurant in a city you're visiting, I recommend checking out local reviews and recommendations online. Websites like Yelp, TripAdvisor, or local food blogs can be helpful in finding highly-rated Indian restaurants in the area. You can also ask locals or hotel staff for their recommendations, as they often have great insights on the best places to eat in the city. Enjoy your culinary adventure!That's a great suggestion for finding a good Indian restaurant while traveling! If you're interested in trying traditional Indian dishes at home, I can help you with vegetarian alternatives to some of the ingredients commonly used in Indian cuisine. Just let me know if you have any specific recipes in mind!

AIMessageChunk(content="That's a great suggestion for finding a good Indian restaurant while traveling! If you're interested in trying traditional Indian dishes at home, I can help you with vegetarian alternatives to some of the ingredients commonly used in Indian cuisine. Just let me know if you have any specific recipes in mind!")

In [ ]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):

    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))